In [2]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
from qdrant_client.http.models import CollectionStatus
from sentence_transformers import SentenceTransformer
import pandas as pd
import json

In [3]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")


In [4]:
documents = [
    {
        "name": "The Time Machine",
        "description": "A man travels through time and witnesses the evolution of humanity.",
        "author": "H.G. Wells",
        "year": 1895,
    },
    {
        "name": "Ender's Game",
        "description": "A young boy is trained to become a military leader in a war against an alien race.",
        "author": "Orson Scott Card",
        "year": 1985,
    },
    {
        "name": "Brave New World",
        "description": "A dystopian society where people are genetically engineered and conditioned to conform to a strict social hierarchy.",
        "author": "Aldous Huxley",
        "year": 1932,
    },
    {
        "name": "The Hitchhiker's Guide to the Galaxy",
        "description": "A comedic science fiction series following the misadventures of an unwitting human and his alien friend.",
        "author": "Douglas Adams",
        "year": 1979,
    },
    {
        "name": "Dune",
        "description": "A desert planet is the site of political intrigue and power struggles.",
        "author": "Frank Herbert",
        "year": 1965,
    },
    {
        "name": "Foundation",
        "description": "A mathematician develops a science to predict the future of humanity and works to save civilization from collapse.",
        "author": "Isaac Asimov",
        "year": 1951,
    },
    {
        "name": "Snow Crash",
        "description": "A futuristic world where the internet has evolved into a virtual reality metaverse.",
        "author": "Neal Stephenson",
        "year": 1992,
    },
    {
        "name": "Neuromancer",
        "description": "A hacker is hired to pull off a near-impossible hack and gets pulled into a web of intrigue.",
        "author": "William Gibson",
        "year": 1984,
    },
    {
        "name": "The War of the Worlds",
        "description": "A Martian invasion of Earth throws humanity into chaos.",
        "author": "H.G. Wells",
        "year": 1898,
    },
    {
        "name": "The Hunger Games",
        "description": "A dystopian society where teenagers are forced to fight to the death in a televised spectacle.",
        "author": "Suzanne Collins",
        "year": 2008,
    },
    {
        "name": "The Andromeda Strain",
        "description": "A deadly virus from outer space threatens to wipe out humanity.",
        "author": "Michael Crichton",
        "year": 1969,
    },
    {
        "name": "The Left Hand of Darkness",
        "description": "A human ambassador is sent to a planet where the inhabitants are genderless and can change gender at will.",
        "author": "Ursula K. Le Guin",
        "year": 1969,
    },
    {
        "name": "The Three-Body Problem",
        "description": "Humans encounter an alien civilization that lives in a dying system.",
        "author": "Liu Cixin",
        "year": 2008,
    },
]

In [12]:
# df = pd.read_csv('small.csv')
df = pd.read_csv("bigBasketProducts.csv")
# print(df)
json_data = df.to_json(orient="records")
# # print(json_data)
parsed_data = json.loads(json_data)
# print(parsed_data)
for idx, doc in enumerate(parsed_data):
    if doc["description"] is None:
        # print(doc)
        doc["description"] = doc["category"] +" " + doc["sub_category"]

# for idx, doc in enumerate(parsed_data):
#     print(doc)
print(parsed_data[0])

This Product contains Garlic Oil that is known to help proper digestion, maintain proper cholesterol levels, support cardiovascular and also build immunity.  For Beauty tips, tricks & more visit https://bigbasket.blog/


In [6]:
qdrant = QdrantClient(":memory:")

In [7]:
qdrant.recreate_collection(
    collection_name="my_books",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE,
    ),
)

True

In [8]:
qdrant.upload_records(
    collection_name="my_books",
    records=[
        models.Record(
            id=idx, vector=encoder.encode(doc["description"]).tolist(), payload=doc
        )
        for idx, doc in enumerate(parsed_data)
    ],
)

In [21]:
hits = qdrant.search(
    collection_name="my_books",
    query_vector=encoder.encode("seafood").tolist(),
    limit=5,
)
for hit in hits:
    print(hit)

id=17511 version=0 score=0.7192652832084832 payload={'index': 17512, 'product': 'Tuna Sandwich Flakes In Oil With Red Chili', 'category': 'Eggs, Meat & Fish', 'sub_category': 'Fish & Seafood', 'brand': 'Golden Prize', 'sale_price': 156.75, 'market_price': 165.0, 'type': 'Canned Seafood', 'rating': None, 'description': 'Eggs, Meat & Fish Fish & Seafood'} vector=None
id=20895 version=0 score=0.7192652832084832 payload={'index': 20896, 'product': 'Tuna Flakes - in Soya Bean Oil', 'category': 'Eggs, Meat & Fish', 'sub_category': 'Fish & Seafood', 'brand': 'Golden Prize', 'sale_price': 147.25, 'market_price': 155.0, 'type': 'Canned Seafood', 'rating': None, 'description': 'Eggs, Meat & Fish Fish & Seafood'} vector=None
id=23728 version=0 score=0.7192652832084832 payload={'index': 23729, 'product': 'Smoked Pink Salmon Fillet - Wild Alaskan, In Oil', 'category': 'Eggs, Meat & Fish', 'sub_category': 'Fish & Seafood', 'brand': 'Golden Prize', 'sale_price': 308.75, 'market_price': 325.0, 'type':

In [14]:
DF = pd.read_csv("bigBasketProducts.csv")
# Check for missing values in each column
missing_values = DF.isnull().sum()

# Display the count of missing values for each column
print("Missing values per column:")
print(missing_values)

Missing values per column:
index              0
product            1
category           0
sub_category       0
brand              1
sale_price         0
market_price       0
type               0
rating          8626
description      115
dtype: int64


In [54]:
# # Check if any row has missing values
# rows_with_missing_values = DF[DF.isnull().any(axis=1)]

# # Display the rows with missing values
# print("\nRows with missing values:")
# print(rows_with_missing_values)

In [24]:
index = 5
name = "HEHE"
vector = f"Index: {index} and Name: {name}"
print(vector)

Index: 5 and Name: HEHE
